In [13]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
from tqdm import tqdm
from balu3.fs.sel    import sfs, clean
from sklearn.linear_model import LogisticRegression
from tensorflow.keras import layers
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from random import randint
from tensorflow.keras.layers import Dropout
from keras.utils import to_categorical
from scipy.stats import binom, betabinom
from tensorflow.keras.preprocessing.sequence import pad_sequences
from joblib import Parallel, delayed
from tensorflow.keras.layers import Masking, LSTM, Dense

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import torch
import keras

In [14]:
#Para la LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Data historica

dfHistorico1 = pd.read_csv("dataHistórica/resultados_pasados_premier.csv")
dfHistorico1 = dfHistorico1.drop("Unnamed: 0",axis = 1)
dfHistorico1 = dfHistorico1.drop(["Position_last_year_home", "Position_last_year_away",
                                  "Points_last_year_home", "Points_last_year_away"],axis = 1) # Para hacer el merge con el segundo.

dfHistorico2 = pd.read_csv("dataHistórica/restoDeLigas.csv")
dfHistorico2 = dfHistorico2.drop("Unnamed: 0",axis = 1)

dfHistorico2['Date'] = pd.to_datetime(dfHistorico2['Date'], dayfirst=True, errors='coerce')
dfHistorico2['Date'] = dfHistorico2['Date'].dt.strftime('%Y-%m-%d')
dfHistorico2 = dfHistorico2[dfHistorico2["Date"] >"2010-01-01"] # Seleccion solo de partidos de 2010 en adelante (porque el otro dataframe tiene info desde ahí)


In [ ]:
dfHistorico = pd.concat([dfHistorico1, dfHistorico2], axis=0)
dfHistorico = dfHistorico.reset_index(drop=True)

In [ ]:
# Data tipo evento

dfEvent = pd.read_csv("dataWhoScored/WhoScoredTeamPerMatchSpatial4x3TimeDiv5.csv")
dfEvent = dfEvent.drop("Unnamed: 0",axis = 1)

In [ ]:
# Ajuste manual de equipos:
# Los que no tengan match son un string vacio.

histToEventNameV1 = {
   "Arsenal": "Arsenal", "Aston Villa": "Aston Villa", "Barnsley": "", "Birmingham City": "Birmingham", "Blackburn": "Blackburn","Blackpool": "Blackpool",
   "Bolton" : "Bolton", "Bournemouth": "Bournemouth", "Bradford City": "", "Brentford": "Brentford", "Brighton": "Brighton", "Burnley": "Burnley", "Cardiff City": "Cardiff",
   "Charlton Ath": "", "Chelsea": "Chelsea", "Coventry City": "", "Crystal Palace": "Crystal Palace", "Derby County":"", "Everton": "Everton", "Fulham": "Fulham",
   "Huddersfield": "Huddersfield", "Hull City": "Hull", "Ipswich Town": "", "Leeds United": "Leeds", "Leicester City": "Leicester", "Liverpool": "Liverpool",
   "Manchester City": "Man City", "Manchester Utd": "Man Utd", "Middlesbrough": "Middlesbrough", "Newcastle Utd": "Newcastle", "Norwich City": "Norwich",
   "Nott'ham Forest": "Nottingham Forest", "Oldham Athletic": "", "Portsmouth": "", "QPR": "QPR", "Reading": "Reading", "Sheffield Utd": "Sheff Utd", "Sheffield Weds": "",
   "Southampton": "Southampton", "Stoke City": "Stoke", "Sunderland": "Sunderland", "Swansea City": "Swansea", "Swindon Town":  "", "Tottenham": "Tottenham", "Watford":"Watford",
   "West Brom": "WBA", "West Ham": "West Ham", "Wigan Athletic": "Wigan", "Wimbledon": "", "Wolves": "Wolves"
}
len(histToEventNameV1)

In [ ]:
histToEventName = {
   "Arsenal": "Arsenal", "Aston Villa": "Aston Villa", "Barnsley": "", "Birmingham City": "Birmingham", "Blackburn": "Blackburn","Blackpool": "Blackpool",
   "Bolton" : "Bolton", "Bournemouth": "Bournemouth", "Bradford City": "", "Brentford": "Brentford", "Brighton": "Brighton", "Burnley": "Burnley", "Cardiff City": "Cardiff",
   "Charlton Ath": "", "Chelsea": "Chelsea", "Coventry City": "", "Crystal Palace": "Crystal Palace", "Derby County":"", "Everton": "Everton", "Fulham": "Fulham",
   "Huddersfield": "Huddersfield", "Hull City": "Hull", "Ipswich Town": "", "Leeds United": "Leeds", "Leicester City": "Leicester", "Liverpool": "Liverpool",
   "Manchester City": "Man City", "Manchester Utd": "Man Utd", "Middlesbrough": "Middlesbrough", "Newcastle Utd": "Newcastle", "Norwich City": "Norwich",
   "Nott'ham Forest": "Nottingham Forest", "Oldham Athletic": "", "Portsmouth": "", "QPR": "QPR", "Reading": "Reading", "Sheffield Utd": "Sheff Utd", "Sheffield Weds": "",
   "Southampton": "Southampton", "Stoke City": "Stoke", "Sunderland": "Sunderland", "Swansea City": "Swansea", "Swindon Town":  "", "Tottenham": "Tottenham", "Watford":"Watford",
   "West Brom": "WBA", "West Ham": "West Ham", "Wigan Athletic": "Wigan", "Wimbledon": "", "Wolves": "Wolves", #V1
   "Aachen":"","Ajaccio":"AC Ajaccio", "Alaves":"Deportivo Alaves", "Almeria": "Almeria", "Amiens":"Amiens","Angers":"Angers", "Arles":"Arles-Avignon", "Ascoli":"",
   "Atalanta":"Atalanta", "Ath Bilbao":"Athletic Club", "Ath Madrid":"Atletico", "Augsburg":"Augsburg", "Auxerre":"Auxerre", "Barcelona":"Barcelona", "Bari":"Bari",
   "Bastia":"SC Bastia", "Bayern Munich":"Bayern", "Benevento":"Benevento", "Betis":"Real Betis", "Bielefeld":"Arminia Bielefeld", "Bochum": "Bochum", "Bologna":"Bologna",
   "Bordeaux":"Bordeaux", "Boulogne": "", "Braunschweig": "Eintracht Braunschweig", "Brescia":"Brescia", "Brest":"Brest", "Cadiz":"Cadiz", "Caen": "Caen", "Cagliari":"Cagliari",
   "Carpi":"AC Carpi", "Catania":"Catania", "Celta":"Celta Vigo", "Cesena": "Cesena", "Chievo":"Chievo", "Clermont":"Clermont Foot", "Cordoba": "Cordoba", "Cottbus":"",
   "Cremonese":"Cremonese", "Crotone":"Crotone", "Darmstadt":"Darmstadt", "Dijon": "Dijon", "Dortmund":"Borussia Dortmund", "Duisburg":"", "Eibar":"Eibar", "Ein Frankfurt":"Eintracht Frankfurt",
   "Elche": "Elche", "Empoli": "Empoli", "Espanol": "Espanyol", "Evian Thonon Gaillard": "Evian", "FC Koln": "FC Koln", "Fiorentina": "Fiorentina", "Fortuna Dusseldorf": "Fortuna Duesseldorf",
   "Freiburg":"Freiburg", "Frosinone": "Frosinone", "Genoa": "Genoa", "Getafe": "Getafe", "Gimnastic":"", "Girona": "Girona", "Granada": "Granada", "Grenoble":"",
   "Greuther Furth": "Greuther Fuerth", "Guingamp": "Guingamp", "Hamburg":"Hamburg", "Hannover":"Hannover", "Hansa Rostock":"", "Heidenheim":"FC Heidenheim", "Hercules":"Hercules",
   "Hertha":"Hertha Berlin", "Hoffenheim":"Hoffenheim", "Huesca": "SD Huesca", "Ingolstadt": "Ingolstadt", "Inter": "Inter", "Juventus":"Juventus", "Kaiserslautern":"Kaiserslautern",
   "Karlsruhe":"", "La Coruna": "Deportivo", "Las Palmas": "Las Palmas", "Lazio": "Lazio", "Le Havre": "Le Havre", "Le Mans":"", "Lecce": "Lecce", "Leganes":"Leganes", "Lens": "Lens",
   "Levante": "Levante", "Leverkusen": "Leverkusen", "Lille":"Lille", "Livorno": "Livorno", "Lorient":"Lorient", "Lyon": "Lyon", "M'gladbach": "Borussia M.Gladbach", "Mainz": "Mainz",
   "Malaga":"Malaga", "Mallorca":"Mallorca", "Messina": "", "Metz": "Metz", "Milan": "AC Milan", "Monaco": "Monaco", "Montpellier": "Montpellier", "Monza": "Monza", "Murcia":"",
   "Nancy":"Nancy", "Nantes":"Nantes", "Napoli":"Napoli", "Nice":"Nice", "Nimes":"Nimes", "Novara": "Novara", "Numancia":"", "Nurnberg":"Nuernberg", "Osasuna": "Osasuna",
   "Paderborn": "Paderborn", "Palermo": "Palermo", "Paris SG": "PSG", "Parma": "Parma Calcio 1913", "Pescara":"Pescara", "RB Leipzig": "RBL","Real Madrid":"Real Madrid", 
   "Recreativo":"", "Reggina":"", "Reims": "Reims", "Rennes":"Rennes", "Roma":"Roma", "Salernitana": "Salernitana", "Sampdoria":"Sampdoria", "Santander": "Racing Santander",
   "Sassuolo":"Sassuolo", "Schalke 04": "Schalke", "Sedan":"", "Sevilla":"Sevilla", "Siena": "Siena", "Sochaux": "Sochaux", "Sociedad": "Real Sociedad", "Sp Gijon":"Sporting Gijon",
   "Spal":"SPAL 2013", "Spezia": "Spezia", "St Etienne":"Saint-Etienne", "St Pauli":"St. Pauli", "Strasbourg":"Strasbourg", "Stuttgart": "Stuttgart", "Tenerife":"", "Torino":"Torino",
   "Toulouse":"Toulouse", "Treviso":"", "Troyes":"Troyes", "Udinese": "Udinese", "Union Berlin":"Union Berlin", "Valencia": "Valencia", "Valenciennes": "Valenciennes", "Valladolid": "Real Valladolid",
   "Vallecano": "Rayo Vallecano", "Venezia":"Venezia", "Verona":"Verona", "Villarreal": "Villarreal", "Werder Bremen": "Werder Bremen", "Wolfsburg": "Wolfsburg", "Xerez": "", "Zaragoza": "Real Zaragoza"
}
len(histToEventName)

In [ ]:
# Para buscar nombres

a = dfHistorico["Team_home"].unique().tolist()
a.sort()
print(a)

In [ ]:
# Para buscar nombres

a = dfEvent["Team"].unique().tolist()
a.sort()
print(a)

In [ ]:
# Nombres de data histórica a nombres de tipo evento.

dfHistorico["Team_home"] = dfHistorico["Team_home"].map(histToEventName)
dfHistorico["Team_away"] = dfHistorico["Team_away"].map(histToEventName)

In [ ]:
# Comprobamos que no esten datos vacios

dfHistorico.isnull().sum() 

In [12]:
# Primero revisamos si estan todos los partidos compatibles, para que sea más robusta la solución

indexHistorical = []
for i in tqdm(range(len(dfHistorico))):
    stringHome = dfHistorico.iloc[i].Team_home
    stringAway = dfHistorico.iloc[i].Team_away
    date = dfHistorico.iloc[i].Date

    dfAux = dfEvent[dfEvent.Date == date]

    dfHome = dfAux[dfAux.Team == stringHome].add_prefix("Home")
    dfAway = dfAux[dfAux.Team == stringAway].add_prefix("Away")

    # dfHome.reset_index(drop=True, inplace=True)
    # dfAway.reset_index(drop=True, inplace=True)
    if len(dfHome) + len(dfAway) == 2: 
        indexHistorical.append(i)
    # concat = pd.concat([dfHome, dfAway], axis = 1)
    # test.append(concat)
    # Buscamos los partidos que tenemos información
    

KeyboardInterrupt: 

In [ ]:
# Creamos el dataframe de partidos a buscar

dfHistoricoCompatible = dfHistorico.iloc[indexHistorical].reset_index(drop=True)
dfHistoricoCompatible = dfHistoricoCompatible.dropna(axis=0)

In [ ]:
dfHistoricoCompatible

In [ ]:
dfHistoricoCompatible.isnull().sum()

In [ ]:
# goals_succ = dfEvent.filter(regex='Goal_Successful_bin').columns.tolist()
# goals_unsucc = dfEvent.filter(regex='Goal_Unsuccessful_bin').columns.tolist()
# goals_succ.extend(goals_unsucc)
# goals = goals_succ

# intervals = ["0-45","45-90"]
# intervals = ["0-15","15-30","30-45","45-60","60-75","75-90"]
intervals = ["0-5","5-10","10-15","15-20","20-25","25-30", "30-35","35-40","40-45",
             "45-50","50-55","55-60","60-65","65-70","70-75", "75-80","80-85","85-90"]

Xdata = []
ydata = []
for i in tqdm(range(len(dfHistoricoCompatible))):
    stringHome = dfHistoricoCompatible.iloc[i].Team_home
    stringAway = dfHistoricoCompatible.iloc[i].Team_away
    date = dfHistoricoCompatible.iloc[i].Date

    # dfAux = dfEvent[dfEvent.Date == date].drop(goals, axis=1) #Test sin los goles
    dfAux = dfEvent[dfEvent.Date == date]

    dfHome = dfAux[dfAux.Team == stringHome].add_prefix("Home")
    dfAway = dfAux[dfAux.Team == stringAway].add_prefix("Away")

    # Dropear valores redundantes
    dfHome = dfHome.drop(["HomeTeam","HomeDate","HomeGoals","Homegame_id"], axis = 1)
    dfAway = dfAway.drop(["AwayTeam","AwayDate","AwayGoals","Awaygame_id"], axis = 1)

    dfHome.reset_index(drop=True, inplace=True)
    dfAway.reset_index(drop=True, inplace=True)

    concat = pd.concat([dfHome, dfAway], axis = 1).reset_index(drop=True)
    x = []
    if len(concat) == 0:
        pass
    else:
        for inter in intervals:
            intervals_columns = concat.filter(regex=f'{inter}$').columns.tolist()
            hist = dfHistoricoCompatible.iloc[i:i+1].drop(["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"], axis=1).values
            x.append(np.append(concat[intervals_columns].values[0], hist))
        ydata.append(dfHistoricoCompatible.iloc[i:i+1]["result"].values[0])
        x = np.array(x)
        Xdata.append(x)

In [15]:
# Para guardar el dataset y no correrlo varias veces 
# np.save("Space4x3Time5.npy", np.array(Xdata))
# np.save("LabelsSpace4x3Time5.npy", np.array(ydata))

Xdata = np.load("Space4x3Time5.npy")
ydata = np.load("LabelsSpace4x3Time5.npy")

# MLP solo datos históricos

In [ ]:
X = dfHistoricoCompatible.drop(["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"], axis=1).values
y = dfHistoricoCompatible.result

# y = to_categorical(y, num_classes=3)

# Dividir los datos en conjuntos de entrenamiento y prueba
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

scaler = MinMaxScaler()
# scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_norm.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])
# Compilar el modelo
with tf.device('GPU:0'): # Usar gpu
    model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=10^(-6))

history = model.fit(X_train_norm, y_train, verbose=1, epochs=100, batch_size=8,
                    validation_data=(X_test_norm, y_test), callbacks=[early_stopping, reduce_lr])

In [ ]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params={"step_size":50,
                        "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax', # "sparsemax" "entmax"
    )
    
clf.fit(X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)], 
    eval_name=['train', 'valid'], 
    eval_metric=['accuracy'], 
    max_epochs=1000 , patience=30, 
    batch_size=28, drop_last=False)

pred = clf.predict(X_test)

# LSTM

In [16]:
# Full data
X = np.array(Xdata)
y = np.array(ydata)
y = to_categorical(y, num_classes=3)


In [17]:
X.shape

(22209, 18, 1270)

In [18]:
y.shape

(22209, 3)

In [19]:
# Dividir los datos en conjuntos de entrenamiento y prueba
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [20]:
# Sin clean
# scaler = MinMaxScaler()
# X_train_reshaped = X_train.reshape(-1, X.shape[2])
# X_train_normalized = scaler.fit_transform(X_train_reshaped)
# X_train_normalized = X_train_normalized.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])

# X_test_reshaped = X_test.reshape(-1, X.shape[2])
# X_test_normalized = scaler.transform(X_test_reshaped)
# X_test_normalized = X_test_normalized.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])


# Con clean 
X_train_reshaped = X_train.reshape(-1, X.shape[2])
X_test_reshaped = X_test.reshape(-1, X.shape[2])

sclean  = clean(X_train_reshaped)      # indices of selected features
X_train_reshaped = deepcopy(X_train_reshaped[:,sclean])
X_test_reshaped = deepcopy(X_test_reshaped[:,sclean])

scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train_reshaped)
X_test_normalized = scaler.transform(X_test_reshaped)

X_train_normalized = X_train_normalized.reshape(X_train.shape[0], X_train.shape[1], len(sclean))
X_test_normalized = X_test_normalized.reshape(X_test.shape[0], X_test.shape[1], len(sclean))


In [21]:
X_train_normalized.shape

(17767, 18, 766)

In [22]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


In [28]:
# Definir el modelo LSTM
model = Sequential()
l2_recurrent_parameter = 1
l_2_kernel_regularizer= 1
# Primera capa LSTM con Dropout y regularización L2
model.add(LSTM(256, return_sequences=True, input_shape=(X_train_normalized.shape[1], X_train_normalized.shape[2]),
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(128,activation="relu"))
model.add(LSTM(256, return_sequences=True,
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(128,activation="relu"))
model.add(LSTM(256, return_sequences=True,
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(128,activation="relu"))
model.add(LSTM(256, return_sequences=False,
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
# Compilación del modelo
with tf.device('GPU:0'): # Usar gpu
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models\InGame",
    monitor='val_loss',
    mode='max',
    save_best_only=True)
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=35, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0000000001)

history = model.fit(X_train_normalized, y_train, verbose=1, epochs=1000, batch_size=16,
                    validation_data=(X_test_normalized, y_test), callbacks=[early_stopping, reduce_lr, model_checkpoint_callback])

Epoch 1/1000


In [26]:
intervals = ["0-5","5-10","10-15","15-20","20-25","25-30", "30-35","35-40","40-45",
             "45-50","50-55","55-60","60-65","65-70","70-75", "75-80","80-85","85-90"]
loss, acc = [], []
for i in range(1, X_test_normalized.shape[1]+1):
    print(intervals[i-1])
    aux = model.evaluate(X_test_normalized[:, :i, :], y_test)
    loss.append(aux[0])
    acc.append(aux[1])

0-5
139/139 [==============================] - 2s 6ms/step - loss: 7.5779 - accuracy: 0.4500
5-10
139/139 [==============================] - 1s 5ms/step - loss: 14.4842 - accuracy: 0.3041
10-15
139/139 [==============================] - 1s 6ms/step - loss: 18.7771 - accuracy: 0.3041
15-20
139/139 [==============================] - 1s 6ms/step - loss: 8.4986 - accuracy: 0.3294
20-25
139/139 [==============================] - 1s 6ms/step - loss: 3.8799 - accuracy: 0.4705
25-30
139/139 [==============================] - 1s 6ms/step - loss: 2.1658 - accuracy: 0.4932
30-35
139/139 [==============================] - 1s 6ms/step - loss: 1.9287 - accuracy: 0.5331
35-40
139/139 [==============================] - 1s 6ms/step - loss: 1.9486 - accuracy: 0.5626
40-45
139/139 [==============================] - 1s 6ms/step - loss: 1.9266 - accuracy: 0.5946
45-50
139/139 [==============================] - 1s 6ms/step - loss: 1.8273 - accuracy: 0.6290
50-55
139/139 [==============================] - 1s

## Pruebas con padding

In [ ]:
import numpy as np
from scipy.stats import betabinom

# Parámetros de la distribución beta-binomial
n = X.shape[1]   # Número de ensayos
a = 0.70*4  # Parámetro alpha de la distribución beta
b = 0.30*4   # Parámetro beta de la distribución beta

# Número de muestras aleatorias que deseas generar
num_samples = X.shape[0]

# Generar números aleatorios de una distribución beta-binomial
samples = betabinom.rvs(n, a, b, size=num_samples)

# Calcular y mostrar la media de las muestras generadas
mean_samples = np.mean(samples)
print(f"Media de las muestras: {mean_samples}")

values, counts = np.unique(samples, return_counts=True)

# Graficar un barplot de las muestras generadas
plt.bar(values, counts / num_samples, color='b', edgecolor='black', alpha=0.7)

# Títulos y etiquetas
plt.title('Histograma de la distribución Beta-Binomial')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')

# Mostrar la gráfica
plt.show()

In [ ]:
# Full data
XFullpad = np.array(Xdata)
y = np.array(ydata)
y = to_categorical(y, num_classes=3)


In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba

split = int(0.8 * XFullpad.shape[0])
XFullpad_train, XFullpad_test = XFullpad[:split], XFullpad[split:]

# # Sin clean
# scaler = MinMaxScaler()
# XFullpad_train_reshaped = XFullpad_train.reshape(-1, X.shape[2])
# XFullpad_train_normalized = scaler.fit_transform(XFullpad_train_reshaped)
# XFullpad_train_normalized = XFullpad_train_normalized.reshape(XFullpad_train.shape[0], XFullpad_train.shape[1], XFullpad_train.shape[2])

# XFullpad_test_reshaped = XFullpad_test.reshape(-1, X.shape[2])
# XFullpad_test_normalized = scaler.transform(XFullpad_test_reshaped)
# XFullpad_test_normalized = XFullpad_test_normalized.reshape(XFullpad_test.shape[0], XFullpad_test.shape[1], XFullpad_test.shape[2])

# Con clean 
XFullpad_train_reshaped = XFullpad_train.reshape(-1, X.shape[2])
XFullpad_test_reshaped = XFullpad_test.reshape(-1, X.shape[2])

sclean  = clean(XFullpad_train_reshaped)      # indices of selected features
XFullpad_train_reshaped = deepcopy(XFullpad_train_reshaped[:,sclean])
XFullpad_test_reshaped = deepcopy(XFullpad_test_reshaped[:,sclean])

scaler = MinMaxScaler()
XFullpad_train_normalized = scaler.fit_transform(XFullpad_train_reshaped)
XFullpad_test_normalized = scaler.transform(XFullpad_test_reshaped)

XFullpad_train_normalized = XFullpad_train_normalized.reshape(XFullpad_train.shape[0], XFullpad_train.shape[1], len(sclean))
XFullpad_test_normalized = XFullpad_test_normalized.reshape(XFullpad_test.shape[0], XFullpad_test.shape[1], len(sclean))



In [ ]:
XFullpad_train_normalized.shape

In [ ]:
# Parámetros de la distribución binomial 
# n = X.shape[1]
# p = 0.9
# size = X.shape[0]
# random_values = binom.rvs(n, p, size=size)

# Parámetros de la distribución beta-binomial
n = XFullpad.shape[1]   # Número de ensayos
size = XFullpad.shape[0]
a = 0.70*4  # Parámetro alpha de la distribución beta
b = 0.30*4   # Parámetro beta de la distribución beta

random_values = betabinom.rvs(n, a, b, size=size)

# random_values = np.random.randint(1, n+1, size)

In [ ]:
#Binom  and uniform
Xpad_train = []
ypad_train = []
for i in range(split):
    Xpad_train.append(XFullpad_train_normalized[i][:random_values[i]+1])
    ypad_train.append(y[i])

Xpad_test = []
ypad_test = []
for i in range(len(XFullpad)-split):
    Xpad_test.append(XFullpad_test_normalized[i][:random_values[i+split]+1])
    ypad_test.append(y[i+split])

# Full
# Xpad = []
# ypad = []
# for i in range(size):
#     for j in range(n):
#         Xpad.append(X[i][:j+1])
#         ypad.append(y[i])

In [ ]:
Xpad_train = pad_sequences(Xpad_train, padding='post', dtype='float64')
Xpad_test = pad_sequences(Xpad_test, padding='post', dtype='float64')
ypad_train = np.array(ypad_train)
ypad_test = np.array(ypad_test)

In [ ]:
ypad_train.shape

In [ ]:
Xpad_train.shape

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Embedding


l2_recurrent_parameter = 0.01
l_2_kernel_regularizer= 0.01

model = Sequential()
model.add(Masking(mask_value=0., input_shape=(Xpad_train.shape[1], Xpad_train.shape[2])))
model.add(LSTM(256, return_sequences=True,
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(128,activation="relu"))
model.add(LSTM(256, return_sequences=True,
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(128,activation="relu"))
model.add(LSTM(256, return_sequences=True,
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(128,activation="relu"))
model.add(LSTM(256, return_sequences=False,
               kernel_regularizer=l2(l_2_kernel_regularizer), recurrent_regularizer=l2(l2_recurrent_parameter)))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
# Compilación del modelo
with tf.device('GPU:0'): # Usar gpu
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath="models\Full",
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True)
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=10^(-9))

history = model.fit(Xpad_train, ypad_train, verbose=1, epochs=1000, batch_size=16,
                    validation_data=(Xpad_test, ypad_test), callbacks=[early_stopping, reduce_lr])

In [ ]:
loss, acc = [], []
for i in range(1, XFullpad_test_normalized.shape[1]+1):
    print(intervals[i])
    aux = model.evaluate(XFullpad_test_normalized[:, :i, :], ypad_test)
    loss.append(aux[0])
    acc.append(aux[1])
print(np.mean(acc))